In [29]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_token_limit=120,
    return_messages=True,
)

In [ ]:
emoji_examples = [
    {
        "movie": "Top Gun",
        "emojis": "🛩️👨‍✈️🔥",
        "result": "Top Gun -> 🛩️👨‍✈️🔥",
    },
    {
        "movie": "The Godfather",
        "emojis": "👨‍👨‍👦🔫🍝",
        "result": "The Godfather -> 👨‍👨‍👦🔫🍝",
    },
    {
        "movie": "Inception",
        "emojis": "💤🧠🌀",
        "result": "Inception -> 💤🧠🌀",
    },
    {
        "movie": "The Matrix",
        "emojis": "🕶️💻👊",
        "result": "The Matrix -> 🕶️💻👊",
    },
]

In [ ]:
emoji_example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=ChatPromptTemplate.from_messages(
        [
            ("human", "영화 {movie}를 나타내는 세 개의 이모티콘은 무엇인가요?"),
            ("ai", "{result}"),
        ]
    ),
    examples=emoji_examples,
)

emoji_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "영화 제목에 대해 그 영화를 나타내는 세 개의 이모티콘을 영화제목과 함께 응답합니다.",
        ),
        emoji_example_prompt,
        (
            "human",
            "영화 {movie}를 나타내는 세 개의 이모티콘은 무엇인가요?",
        ),
    ]
)

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

In [ ]:
def load_memory(_):
    return memory.load_memory_variables({})["history"]


emoji_chain = emoji_prompt | chat
chat_chain = RunnablePassthrough.assign(history=load_memory) | chat_prompt | chat


def invoke_chat_chain(question):
    result = chat_chain.invoke({"question": question})
    memory.save_context({"input": question}, {"output": result.content})


def invoke_emoji_chain(movie):
    result = emoji_chain.invoke({"movie": movie})
    memory.save_context({"input": movie}, {"output": result.content})

In [ ]:
invoke_emoji_chain("Interstellar")
print("\n\n")
invoke_emoji_chain("트루먼쇼")
print("\n\n")
invoke_emoji_chain("Darknight")
print("\n\n")
invoke_chat_chain("첫번째로 물어본 영화 제목이 뭐야?")
print("\n\n")
invoke_chat_chain("마지막으로 물어본 영화 제목이 뭐야?")
print("\n\n")
invoke_chat_chain("두번째로 물어본 영화 제목이 뭐야?")